## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import requests
import gmaps
import pandas as pd
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Russell,US,32.3502,-85.1999,83.48,64,20,6.91,few clouds
1,Saint-Augustin,CA,51.2260,-58.6502,46.69,94,99,13.51,overcast clouds
2,Barrow,US,71.2906,-156.7887,30.22,80,90,14.97,light snow
3,Batagay-Alyta,RU,67.8006,130.4114,15.10,87,9,3.56,clear sky
4,Galle,LK,6.0367,80.2170,78.42,84,39,8.16,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 50
What is your desired maximum temperature for your trip? 70


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
desired_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &
                                       (city_data_df["Max Temp"] <= max_temp)]
desired_cities_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
11,Crotone,IT,39.0851,17.1178,68.90,69,79,2.82,broken clouds
13,Cape Town,ZA,-33.9258,18.4232,55.09,63,40,4.61,scattered clouds
14,Iglesias,IT,39.3114,8.5350,68.76,82,40,2.98,scattered clouds
16,Port Lincoln,AU,-34.7333,135.8667,55.83,82,100,27.09,overcast clouds
19,Jamestown,US,42.0970,-79.2353,69.19,60,1,1.99,clear sky
...,...,...,...,...,...,...,...,...,...
656,Lianran,CN,24.9227,102.4850,63.14,97,4,2.93,clear sky
667,Halifax,CA,44.6453,-63.5724,57.69,79,84,1.14,broken clouds
668,Carahue,CL,-38.7000,-73.1667,52.00,76,54,5.37,broken clouds
671,Khasan,RU,42.4308,130.6434,63.45,81,82,10.83,light rain


In [5]:
# 4a. Determine if there are any empty rows.
desired_cities_df.isnull().sum()

City                   0
Country                6
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = desired_cities_df.dropna()
clean_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
11,Crotone,IT,39.0851,17.1178,68.90,69,79,2.82,broken clouds
13,Cape Town,ZA,-33.9258,18.4232,55.09,63,40,4.61,scattered clouds
14,Iglesias,IT,39.3114,8.5350,68.76,82,40,2.98,scattered clouds
16,Port Lincoln,AU,-34.7333,135.8667,55.83,82,100,27.09,overcast clouds
19,Jamestown,US,42.0970,-79.2353,69.19,60,1,1.99,clear sky
...,...,...,...,...,...,...,...,...,...
656,Lianran,CN,24.9227,102.4850,63.14,97,4,2.93,clear sky
667,Halifax,CA,44.6453,-63.5724,57.69,79,84,1.14,broken clouds
668,Carahue,CL,-38.7000,-73.1667,52.00,76,54,5.37,broken clouds
671,Khasan,RU,42.4308,130.6434,63.45,81,82,10.83,light rain


In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Crotone,IT,68.90,broken clouds,39.0851,17.1178,
13,Cape Town,ZA,55.09,scattered clouds,-33.9258,18.4232,
14,Iglesias,IT,68.76,scattered clouds,39.3114,8.5350,
16,Port Lincoln,AU,55.83,overcast clouds,-34.7333,135.8667,
19,Jamestown,US,69.19,clear sky,42.0970,-79.2353,
21,Mataura,NZ,52.48,overcast clouds,-46.1927,168.8643,
24,Port Elizabeth,ZA,51.10,light intensity shower rain,-33.9180,25.5701,
30,Torbay,CA,57.27,broken clouds,47.6666,-52.7314,
31,Maarianhamina,AX,53.46,clear sky,60.0973,19.9348,
32,Hobart,AU,56.12,broken clouds,-42.8794,147.3294,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Cannot find hotel...skipping.")    
        

Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.
Cannot find hotel...skipping.


In [16]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Crotone,IT,68.90,broken clouds,39.0851,17.1178,Helios Hotel
13,Cape Town,ZA,55.09,scattered clouds,-33.9258,18.4232,Southern Sun Waterfront Cape Town
14,Iglesias,IT,68.76,scattered clouds,39.3114,8.5350,"B&B Mare Monti Miniere - Sardinia sud-ovest, t..."
16,Port Lincoln,AU,55.83,overcast clouds,-34.7333,135.8667,Port Lincoln Hotel
19,Jamestown,US,69.19,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
...,...,...,...,...,...,...,...
656,Lianran,CN,63.14,clear sky,24.9227,102.4850,半岛酒店
667,Halifax,CA,57.69,broken clouds,44.6453,-63.5724,Hotel Halifax
668,Carahue,CL,52.00,broken clouds,-38.7000,-73.1667,Hospedaje Familiar
671,Khasan,RU,63.45,light rain,42.4308,130.6434,


In [17]:
# 7. Drop the rows where there is no Hotel Name.
D = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = D.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Crotone,IT,68.90,broken clouds,39.0851,17.1178,Helios Hotel
13,Cape Town,ZA,55.09,scattered clouds,-33.9258,18.4232,Southern Sun Waterfront Cape Town
14,Iglesias,IT,68.76,scattered clouds,39.3114,8.5350,"B&B Mare Monti Miniere - Sardinia sud-ovest, t..."
16,Port Lincoln,AU,55.83,overcast clouds,-34.7333,135.8667,Port Lincoln Hotel
19,Jamestown,US,69.19,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
...,...,...,...,...,...,...,...
651,Sicuani,PE,60.85,overcast clouds,-14.2694,-71.2261,Wilkamayu Hotel
656,Lianran,CN,63.14,clear sky,24.9227,102.4850,半岛酒店
667,Halifax,CA,57.69,broken clouds,44.6453,-63.5724,Hotel Halifax
668,Carahue,CL,52.00,broken clouds,-38.7000,-73.1667,Hospedaje Familiar


In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 8a. Create the output File (CSV)
output_data_file = "weatherpy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig